In [1]:
import os
import csv
import cv2
import math
import time
import json
import glob
import shutil
import seaborn as sn
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm as tqdm
from tabulate import tabulate

In [2]:
Match_result_csv_path = '../csv_file/PBL_match_result_all_cej_bone_fold1.csv'
Match_result_csv = pd.read_csv(Match_result_csv_path)

show_csv = Match_result_csv[['image_name','tooth_side','stage','PBL_predict_tooth','PBL_value','PBL_stage','tooth_path']]

In [3]:
Match_result_csv.head()

,Unnamed: 0,image_name,tooth_side,stage,PBL_predict_tooth,PBL_value,PBL_stage,tooth_path,CEJ_points,Bone_level_points
0,0,NN_190619_104716_EA3E9,9M,1.0,9_L,12.255,1,NN_190619_104716_EA3E9_0.PNG,1,1
1,1,NN_190619_104716_EA3E9,9D,1.0,9_R,9.233,1,NN_190619_104716_EA3E9_0.PNG,1,1
2,2,NN_190619_104716_EA3E9,10M,3.0,10_L,27.999,2,NN_190619_104716_EA3E9_1.PNG,1,1
3,3,NN_190619_104716_EA3E9,10D,3.0,10_R,70.792,3,NN_190619_104716_EA3E9_1.PNG,1,1
4,4,NN_190619_104720_EA3E7,7D,1.0,7_L,32.519,2,NN_190619_104720_EA3E7_1.PNG,1,1


In [4]:
stage_set = list(set(Match_result_csv['stage']))
print(stage_set)
PBL_stage_set = list(set(Match_result_csv['PBL_stage']))
print(PBL_stage_set)

[0.0, 1.0, 2.0, 3.0]
['3', nan, '-999', 'No_match', '0', '1', '2']


In [15]:
folder_path = '../'
o_image_folder = os.path.join(folder_path,'1_o_image')
o_image_path_list = glob.glob(o_image_folder+'/*.PNG')
o_image_path_list.sort()

s_image_folder = os.path.join(folder_path,'1_seg_image')
s_image_path_list = glob.glob(s_image_folder+'/*.PNG')
s_image_path_list.sort()

seg_image_folder = os.path.join(folder_path,'3_pattern_detection_result')
seg_image_point_path_list = glob.glob(seg_image_folder+'/*.PNG')
seg_image_point_path_list.sort()

seg_fill_image_folder = os.path.join(folder_path,'2_seg_img_with_cej')
seg_fill_image_point_path_list = glob.glob(seg_fill_image_folder+'/*.PNG')
seg_fill_image_point_path_list.sort()

#seg_image_folder = os.path.join(folder_path,'4_result_with_label')
seg_image_folder = os.path.join(folder_path,'3_pattern_detection_result')
seg_image_info_path_list = glob.glob(seg_image_folder+'/*.PNG')
seg_image_info_path_list.sort()

json_save_path = '../../../json_file/*.json'
json_file_list = glob.glob(json_save_path)
print(len(json_file_list))

print(len(o_image_path_list),len(seg_image_info_path_list),len(seg_image_point_path_list))

2973
0 0 0


In [6]:
## PBL_stage,CEJ_points,Bone_level_points should be string
target_case = {'image_name':None,'stage':3,'PBL_stage':None,'CEJ_points':None,'Bone_level_points':None}

In [7]:
if target_case['stage'] == 3:
    PBL_value_range = [(0, 27)]
elif target_case['stage'] == 2:
    PBL_value_range = [(0, 9), (39,1000)]
elif target_case['stage'] == 1:
    PBL_value_range = [(21, 1000)]
elif target_case['stage'] == 0:
    PBL_value_range = [(21, 1000)]

In [8]:
Match_result_csv.head()

,Unnamed: 0,image_name,tooth_side,stage,PBL_predict_tooth,PBL_value,PBL_stage,tooth_path,CEJ_points,Bone_level_points
0,0,NN_190619_104716_EA3E9,9M,1.0,9_L,12.255,1,NN_190619_104716_EA3E9_0.PNG,1,1
1,1,NN_190619_104716_EA3E9,9D,1.0,9_R,9.233,1,NN_190619_104716_EA3E9_0.PNG,1,1
2,2,NN_190619_104716_EA3E9,10M,3.0,10_L,27.999,2,NN_190619_104716_EA3E9_1.PNG,1,1
3,3,NN_190619_104716_EA3E9,10D,3.0,10_R,70.792,3,NN_190619_104716_EA3E9_1.PNG,1,1
4,4,NN_190619_104720_EA3E7,7D,1.0,7_L,32.519,2,NN_190619_104720_EA3E7_1.PNG,1,1


In [9]:
# print(PBL_value_range[1][0],PBL_value_range[1][1])

In [10]:
target_table = Match_result_csv

for key in target_case:
    if target_case[key] != None:
        target_table = target_table.loc[(target_table[key] == target_case[key])]

if target_case['stage']==2:
    type_ = 0# 0:(0%~9%) 1:(>39%)
    target_table['PBL_value'] = pd.to_numeric(target_table['PBL_value'], errors='coerce')
    target_table = target_table.dropna()
#     show_table1 = target_table.loc[(target_table['PBL_value'] > PBL_value_range[0][0]) & (target_table['PBL_value'] < PBL_value_range[0][1])]
#     show_table2 = target_table.loc[(target_table['PBL_value'] > PBL_value_range[1][0]) & (target_table['PBL_value'] < PBL_value_range[1][1])]
#     show_table = pd.concat([show_table1, show_table2])
    show_table = target_table.loc[(target_table['PBL_value'] > PBL_value_range[type_][0]) & (target_table['PBL_value'] < PBL_value_range[type_][1])]
else:
    target_table['PBL_value'] = pd.to_numeric(target_table['PBL_value'], errors='coerce')
    target_table = target_table.dropna()
    show_table = target_table.loc[(target_table['PBL_value'] > PBL_value_range[0][0]) & (target_table['PBL_value'] < PBL_value_range[0][1])]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [11]:

target_table = show_table
target_table_img_name = target_table['image_name']
target_name_list = list(set(list(target_table_img_name)))
target_name_list.sort()
print(len(target_name_list))

# 0:11張14邊 tooth,  1:166張220邊 tooth,  2: low74張86邊tooth  , high 39張53邊 tooth ,  3:64張82邊 tooth

56


In [14]:
target_table_img_name = target_table['image_name']
target_name_list = list(set(list(target_table_img_name)))
target_name_list.sort()
print(len(target_name_list))
target_name_list = target_name_list[:]

#assert target_case['PBL_stage'] != 'No_match','Cannot be No_match'
count = 0

if target_case['PBL_stage']!='No_match':
    for index,image_name in tqdm(enumerate(target_name_list),total=len(target_name_list)):
        print(index,":",image_name)
        Match_tooth_json = [json_path for json_path in json_file_list if image_name in json_path]
        target_o = [img_path for img_path in o_image_path_list if image_name in img_path]
        target_s = [img_path for img_path in s_image_path_list if image_name in img_path]
        target_info = [img_path for img_path in seg_image_info_path_list if image_name in img_path]
        target_pattern = [img_path for img_path in seg_fill_image_point_path_list if image_name in img_path]
        
        target_csv = show_csv.loc[(show_csv['image_name'] == image_name)]
        target_csv = target_csv.loc[(target_csv['stage'] == target_case['stage'])]
        target_csv['PBL_value'] = pd.to_numeric(target_csv['PBL_value'], errors='coerce')
        target_csv = target_csv.dropna()
#         if target_case['stage']==2 and type_==1:
#             target_csv = target_csv.loc[(target_csv['PBL_value'] > PBL_value_range[1][0]) & (target_csv['PBL_value'] < PBL_value_range[1][1])]
#         else:
#             target_csv = target_csv.loc[(target_csv['PBL_value'] > PBL_value_range[0][0]) & (target_csv['PBL_value'] < PBL_value_range[0][1])]
# #         target_table = show_table
        target_csv = target_csv[['tooth_side','stage','PBL_predict_tooth','PBL_value','PBL_stage','tooth_path']]
        print(tabulate(target_csv, headers='keys', tablefmt='psql'))
        count+=len(target_csv)
        if Match_tooth_json != []:
            tooth_in_json_list = []
            with open(Match_tooth_json[0], 'r') as file:
                json_file = json.load(file)
                for tooth in json_file['shapes']:
                    title = tooth['label']
                    tooth_in_json_list.append(title)
            print('Root in json:',tooth_in_json_list)
        print(target_o)
        if target_o != []:
            o_img = cv2.imread(target_o[0],0)
            s_img = cv2.imread(target_s[0])
            plt.figure(figsize=(12,12))
            plt.subplot(121)
            plt.imshow(o_img,cmap='gray')
            plt.subplot(122)
            plt.imshow(cv2.cvtColor(s_img,cv2.COLOR_BGR2RGB))
            plt.show()
        show_img_list = list(target_csv['tooth_path'])
        for p_img_name in show_img_list:
            blank_image = np.zeros((o_img.shape[0], o_img.shape[1], 3), np.uint8)
            target_info = [img_path for img_path in seg_image_info_path_list if p_img_name[:-4] in img_path]
            target_pattern = [img_path for img_path in seg_fill_image_point_path_list if p_img_name[:-4] in img_path]
            target_info_img = blank_image.copy()
            target_pattern_img = blank_image.copy()
            if target_info!=[]:
                target_info_img = cv2.cvtColor(cv2.imread(target_info[0]),cv2.COLOR_BGR2RGB)
            if target_pattern!=[]:
                target_pattern_img = cv2.cvtColor(cv2.imread(target_pattern[0]),cv2.COLOR_BGR2RGB) 
            plt.figure(figsize=(12,21))
            plt.subplot(121)
            plt.imshow(target_info_img)
            plt.subplot(122)
            plt.imshow(target_pattern_img)
            plt.show()
            
print(count)

56


0 : NN_071019_103621_C073B3
+------+--------------+---------+---------------------+-------------+-------------+-------------------------------+
|      | tooth_side   |   stage | PBL_predict_tooth   |   PBL_value |   PBL_stage | tooth_path                    |
|------+--------------+---------+---------------------+-------------+-------------+-------------------------------|
| 9941 | 14D          |       3 | 14_R                |       0     |           0 | NN_071019_103621_C073B3_2.PNG |
| 9943 | 15D          |       3 | 15_R                |       8.155 |           1 | NN_071019_103621_C073B3_3.PNG |
+------+--------------+---------+---------------------+-------------+-------------+-------------------------------+
Root in json: ['13_LR', '14_LR', '15_LR']
[]


NameError: name 'o_img' is not defined

In [ ]:
img_path_list = glob.glob('/root/notebooks/tooth_thsis_experiment/PBL_detection/Data/atient_folder/*/*/*.PNG')
print(len(img_path_list))

In [ ]:
for path in tqdm(img_path_list, total=len(img_path_list)):
    os.remove(path)

In [ ]:
# json_list = glob.glob('/root/notebooks/tooth_thsis_experiment/PBL_detection/Data/atient_folder/*/*/*.json')
# print(len(json_list))

# # json_list = glob.glob('/root/notebooks/tooth_thsis_experiment/PBL_detection/Data/patient_folder/*/*/*.json')
# # print(len(json_list))

In [ ]:
# save_path = '/root/notebooks/tooth_thsis_experiment/PBL_detection/Data/json_file/'

# for path in tqdm(json_list, total=len(json_list)):
#     json_name = path.split('/')[-1]
#     shutil.copyfile(path, os.path.join(save_path, json_name))

In [ ]:
2977

In [ ]:
# json_list = glob.glob('/root/notebooks/tooth_thsis_experiment/PBL_detection/Data/json_file/*.json')
# print(len(json_list))